# Infer using a Transformer

> Module for training on a dataset of embeddings

In [ ]:
# | default_exp models.inference

In [ ]:
%reload_ext nb_black
%reload_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import sys

__root = "../../"
sys.path.append(__root)

In [ ]:
# | export
from clip_video_classifier.preprocess.video_to_frames import (
    video_to_frames,
    get_video_duration,
)
from clip_video_classifier.models.transformer_encoder import (
    TransformerEncoder,
    collate_fn,
)
from clip_video_classifier.models.frame_embeddings import Frame2Embeddings
from clip_video_classifier.data.dataset import ClipEmbeddingsDataset
from transformers import Trainer, TrainingArguments
from torch_snippets import *
from functools import lru_cache
from torch_snippets.trainer import to
from clip_video_classifier.preprocess.create_annotations import wget
from transformers import Trainer, TrainingArguments

In [ ]:
# | export


def download_trained_model(model_path):
    makedir(parent(model_path))
    if not exists(model_path):
        wget(
            "https://huggingface.co/sizhkhy/ssbd-video-classifier/resolve/main/pytorch_model.bin",
            model_path,
        )
    model = TransformerEncoder(4, 512, 128).cpu()
    load_torch_model_weights_to(model, model_path, device="cpu")
    model.eval()
    return model


class Inference:
    def __init__(self, model_path):
        self.model, self.clip = self.load_model_and_clip(model_path)

    def load_model_and_clip(self, model_path):
        model = download_trained_model(model_path)
        clip = Frame2Embeddings(device="cpu")
        return model, clip

    @torch.no_grad()
    def predict_on_video_path(self, video_path, start_sec=None):
        if start_sec is None:
            duration = get_video_duration(video_path)
            output = {}
            for start in range(0, int(duration), 5):
                output[start] = self.predict_on_video_path(video_path, start_sec=start)
            return output
        frames = video_to_frames(
            video_path, frames_path=None, start_sec=0, clip_duration=5, verbose=True
        )
        frames = to(frames, self.clip.device)
        input = self.clip.frames2clip_image_embeddings(frames)
        input = {"embeddings": input}
        Info(f"{input=}")
        input = collate_fn([input])
        predictions = self.model(**input)
        predictions = F.softmax(predictions["logits"], dim=-1)
        Info(f"{predictions=}")
        confidence, predicted_class = predictions.max(1)
        predicted_class = ClipEmbeddingsDataset.id2label[predicted_class.item()]
        confidence = confidence.item()
        Info(f"{predicted_class=} @ {confidence=}")
        return {
            "predicted_class": predicted_class,
            "confidence": f"{confidence:.2f}",
        }

In [ ]:
inference = Inference(
    "/mnt/347832F37832B388/projects/ssbd/cogniable-assignment/artifacts/a/pytorch_model.bin2"
)
video_path = (
    "/mnt/347832F37832B388/ml-datasets/ssbd/ssbd-raw-videos/v_ArmFlapping_09.mp4"
)
content = inference.predict_on_video_path(video_path)


Usage

```python
inference = Inference("/mnt/347832F37832B388/projects/ssbd/cogniable-assignment/artifacts/a/pytorch_model.bin")
video_path = "/mnt/347832F37832B388/ml-datasets/ssbd/ssbd-raw-videos/v_ArmFlapping_09.mp4"
content = inference.predict_on_video_path(video_path)
```

In [ ]:
import nbdev

nbdev.nbdev_export()